In [ ]:
#| default_exp page

This notebook implements pagination for the Stripe API.

In [ ]:
#| export
from fastcore.all import *
from faststripe.core import *

In [ ]:
#| hide
from nbdev import *

import itertools, os

In [ ]:
sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

In [ ]:
sapi.customers.get

[customers.get](https://docs.stripe.com/api/customers/list)(created: 'str', email: 'str', ending_before: 'str', expand: 'str', limit: 'str', starting_after: 'str', test_clock: 'str'): *List all customers*

In [ ]:
customers = sapi.customers.get(limit = 1)
customers.keys(), customers.data[0].keys()


(
    dict_keys(['object', 'data', 'has_more', 'url']),
    dict_keys(['id', 'object', 'address', 'balance', 'created', 'currency', 'default_source', 'delinquent', 'description', 'discount', 'email', 'invoice_prefix', 'invoice_settings', 'livemode', 'metadata', 'name', 'next_invoice_sequence', 'phone', 'preferred_locales', 'shipping', 'tax_exempt', 'test_clock'])
)

The `paged` function provides an iterator that automatically handles pagination for Stripe API list methods.

In [ ]:
#| export
def paged(oper, *args, **kwargs):
    """Iterate through all pages of a Stripe API operation."""
    page = oper(*args, **kwargs)
    yield page
    while page.has_more:
        last_id = page.data[-1].id
        page = oper(*args, starting_after=last_id, **kwargs)
        yield page

In [ ]:
ps = L(itertools.islice(paged(sapi.customers.get, limit=2), 2))
cs = L(c for p in ps for c in p.data)
test_eq(len(ps), 2)
test_eq(len(cs), 4)

Number of pages: 2


The `pages` function retrieves all items from all pages and returns them as a single list.

In [ ]:
#| export
def pages(oper, *args, **kwargs):
    """Retrieve all items from all pages of a Stripe API operation."""
    return L(p for page in paged(oper, *args, **kwargs) for p in page.data)

Let's test the pagination with the `customers` endpoint.

In [ ]:
customers = pages(sapi.customers.get, limit=100)
len(customers), customers[0].keys()


(
    588,
    dict_keys(['id', 'object', 'address', 'balance', 'created', 'currency', 'default_source', 'delinquent', 'description', 'discount', 'email', 'invoice_prefix', 'invoice_settings', 'livemode', 'metadata', 'name', 'next_invoice_sequence', 'phone', 'preferred_locales', 'shipping', 'tax_exempt', 'test_clock'])
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()